**1)HOUSE PRICE  PREDICTION**

In [ ]:
pip install streamlit scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 48.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd

data = pd.read_csv("HousingData.csv")
print(data.columns)


Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')


In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# ---------------------------
# Load Dataset
# ---------------------------
data = pd.read_csv("HousingData.csv")   # make sure file is uploaded or in same folder

# Features and Target
X = data.drop("MEDV", axis=1)
y = data["MEDV"]

# Handle missing data
X = X.fillna(X.mean())

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Model trained ✅ | MSE: {mse:.2f}")

# ---------------------------
# Gradio Interface
# ---------------------------

# Function for prediction
def predict_house_price(*features):
    # Convert inputs to DataFrame
    input_data = pd.DataFrame([features], columns=X.columns)
    # Scale input
    input_scaled = scaler.transform(input_data)
    # Predict
    prediction = model.predict(input_scaled)[0]
    return f"Predicted Price: ${prediction:.2f} (in $1000s)"

# Create input fields dynamically for each feature
inputs = []
for feature in X.columns:
    min_val = float(X[feature].min())
    max_val = float(X[feature].max())
    mean_val = float(X[feature].mean())
    inputs.append(gr.Number(label=feature, value=mean_val))

# Gradio Interface
demo = gr.Interface(
    fn=predict_house_price,
    inputs=inputs,
    outputs="text",
    title="🏠 House Price Prediction",
    description=f"Predicts median house price (MEDV). Model MSE = {mse:.2f}"
)

demo.launch()


Model trained ✅ | MSE: 25.02
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://36cf75130e9fe0d5eb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ---------------------------
# Load Dataset
# ---------------------------
data = pd.read_csv("HousingData.csv")   # make sure the file is in the same folder

# Features and Target
X = data.drop("MEDV", axis=1)
y = data["MEDV"]

# Handle missing data
X = X.fillna(X.mean())

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# ---------------------------
# Regression Metrics
# ---------------------------
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n--- Regression Metrics ---")
print(f"MSE  : {mse:.2f}")
print(f"MAE  : {mae:.2f}")
print(f"R²   : {r2:.2f}")

# ---------------------------
# Convert Regression → Classification
# ---------------------------
# Define bins (cover full range of y, including negatives if any)
bins = [y.min() - 1, 20, 35, np.inf]   # ensure all values fall into bins
labels = ["Low", "Medium", "High"]

# Convert continuous target & predictions to categories
y_test_class = pd.cut(y_test, bins=bins, labels=labels)
y_pred_class = pd.cut(y_pred, bins=bins, labels=labels)

# Drop NaN rows (if any values fall outside bins)
mask = ~y_test_class.isna() & ~y_pred_class.isna()
y_test_class = y_test_class[mask]
y_pred_class = y_pred_class[mask]

# Encode labels to numeric
encoder = LabelEncoder()
encoder.fit(labels)

y_test_encoded = encoder.transform(y_test_class)
y_pred_encoded = encoder.transform(y_pred_class)

# Classification Metrics
print(f"\n--- Classification Metrics ---")
print("Accuracy :", accuracy_score(y_test_encoded, y_pred_encoded))
print("Precision:", precision_score(y_test_encoded, y_pred_encoded, average="weighted"))
print("Recall   :", recall_score(y_test_encoded, y_pred_encoded, average="weighted"))
print("F1-score :", f1_score(y_test_encoded, y_pred_encoded, average="weighted"))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_encoded, y_pred_encoded))
print("\nClassification Report:\n", classification_report(y_test_encoded, y_pred_encoded, target_names=labels))

# ---------------------------
# Gradio Interface
# ---------------------------
def predict_house_price(*features):
    input_data = pd.DataFrame([features], columns=X.columns)
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]

    # Convert to category
    category = pd.cut([prediction], bins=bins, labels=labels)[0]

    return f"Predicted Price: ${prediction:.2f} (in $1000s)\nCategory: {category}"

# Create input fields dynamically
inputs = []
for feature in X.columns:
    mean_val = float(X[feature].mean())
    inputs.append(gr.Number(label=feature, value=mean_val))

demo = gr.Interface(
    fn=predict_house_price,
    inputs=inputs,
    outputs="text",
    title="🏠 House Price Prediction",
    description=f"Regression MSE = {mse:.2f}, R² = {r2:.2f}\nClassification used for Low/Medium/High categories."
)

demo.launch()



--- Regression Metrics ---
MSE  : 25.02
MAE  : 3.15
R²   : 0.66

--- Classification Metrics ---
Accuracy : 0.83
Precision: 0.8375026833631486
Recall   : 0.83
F1-score : 0.8292292587137947

Confusion Matrix:
 [[ 4  0  3]
 [ 0 39  9]
 [ 1  4 40]]

Classification Report:
               precision    recall  f1-score   support

         Low       0.80      0.57      0.67         7
      Medium       0.91      0.81      0.86        48
        High       0.77      0.89      0.82        45

    accuracy                           0.83       100
   macro avg       0.83      0.76      0.78       100
weighted avg       0.84      0.83      0.83       100

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://258a17684f084119b0.gra

**SPAM MAIL DETECTION**

In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# ---------------------------
# Load Dataset
# ---------------------------
data = pd.read_csv("spam_ham_dataset.csv")  # replace with your dataset filename

# Features and Target
X = data["text"]
y = data["label"]   # or use 'label_num' if already numeric

# Encode labels to numeric if needed
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# ---------------------------
# Text Preprocessing using TF-IDF
# ---------------------------
vectorizer = TfidfVectorizer(stop_words='english', max_features=3000)  # limit to top 3000 words
X_tfidf = vectorizer.fit_transform(X)

# ---------------------------
# Train-test split
# ---------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y_encoded, test_size=0.2, random_state=42
)

# ---------------------------
# Train Naive Bayes Model
# ---------------------------
model = MultinomialNB()
model.fit(X_train, y_train)

# ---------------------------
# Classification Metrics
# ---------------------------
y_pred = model.predict(X_test)

print(f"\n--- Classification Metrics ---")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average="weighted"))
print("Recall   :", recall_score(y_test, y_pred, average="weighted"))
print("F1-score :", f1_score(y_test, y_pred, average="weighted"))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=encoder.classes_))

# ---------------------------
# Gradio Interface
# ---------------------------
def predict_spam(message):
    # Transform the input message using the same TF-IDF vectorizer
    msg_tfidf = vectorizer.transform([message])
    prediction = model.predict(msg_tfidf)[0]
    label = encoder.inverse_transform([prediction])[0]
    return f"This message is classified as: {label.upper()}"

demo = gr.Interface(
    fn=predict_spam,
    inputs=gr.Textbox(label="Enter email message"),
    outputs="text",
    title="📧 Spam Mail Detection using Naive Bayes",
    description="Enter a message to classify as HAM (not spam) or SPAM.\nModel uses TF-IDF + MultinomialNB."
)

demo.launch()



--- Classification Metrics ---
Accuracy : 0.9449275362318841
Precision: 0.9464348470925827
Recall   : 0.9449275362318841
F1-score : 0.9453847504718782

Confusion Matrix:
 [[705  37]
 [ 20 273]]

Classification Report:
               precision    recall  f1-score   support

         ham       0.97      0.95      0.96       742
        spam       0.88      0.93      0.91       293

    accuracy                           0.94      1035
   macro avg       0.93      0.94      0.93      1035
weighted avg       0.95      0.94      0.95      1035

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://343a3bc0ba32e2ac7e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio dep